In [0]:
import os
import re
import sys
import ssl
import json
import nltk
import time
import math
import string
import pickle
import argparse

import torch
import numpy as np
import pandas as pd
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from collections import Counter, OrderedDict
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence

In [0]:
import warnings
# warnings.filterwarnings('ignore')

warnings.filterwarnings(action='once')

In [0]:
# trn = pd.read_pickle('jigsaw_toxic/20pct_model_35toxic_train.pkl')
# val = pd.read_pickle('jigsaw_toxic/20pct_model_35toxic_val.pkl')

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from google.colab import files
uploaded = files.upload()

Saving exec_lstm.py to exec_lstm (1).py


In [5]:
import exec_lstm as exl

Using CUDA? True cuda:0


In [6]:
torch.cuda.is_available()

True

In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
# !unzip jigsaw-unintended-bias-in-toxicity-classification.zip

In [0]:
# X_train = trn.drop('label', axis=1)
# y_train = trn['label']
# test_sample = val.sample( n=math.ceil(len(X_train)*0.2), random_state=1008 )
# X_test = test_sample.drop('label', axis=1)
# y_test = test_sample['label'] 

In [0]:
# lstm_model = exl.LSTMModel(X_train[:500], y_train[:500], X_test[:100], y_test[:100], 
#                            hidden_dim=50, num_layers=1, embed_dim=50, batch_size=1,
#                            dropout=0, num_classes=2)

In [0]:
# if exl.USE_CUDA:
#     print(exl.USE_CUDA)
#     lstm_model = lstm_model.cuda()

In [0]:
# lstm_model.train()

In [0]:
# NUM_EPOCHS = 6
# hist_lstm = np.zeros(NUM_EPOCHS)
# model_name="toy_dataset"

In [0]:
# results, model_state_dict = lstm_model.run_model(y_train[:500], X_test[:100], y_test[:100], NUM_EPOCHS, 
#                                                  hist_lstm, text_col='cleaned_no_stem', savestate=model_name) 

In [0]:
# exl.get_metrics(results, should_print=True, detailed = False, 
#                 label_col="y_true", score_col="predicted_score")

### Creating Batch Loader to Enable Larger Dataset Training

In [0]:
from google.colab import auth
auth.authenticate_user()

In [17]:
!pip install gcsfs

In [0]:
train_full = pd.read_csv('gs://no-hate/train.csv')

In [0]:
# train_full = pd.read_csv("train.csv")

In [0]:
train_full['label'] = train_full.target.apply(lambda x: 1 if x > 0.5 else 0)

In [0]:
test_ratio = 0.2

In [564]:
train_set, val_set, test_set = exl.get_samples(train_full, proportion=0.2, 
                                                  train_test_ratio=(1-test_ratio))

Retrieving training samples at proportion 0.2


In [25]:
print(train_sample.shape, val_set.shape, test_set.shape)

(288752, 47) (180419, 47) (180694, 47)


In [326]:
train_sample.loc[train_sample.comment_text != ''].shape

(288752, 47)

In [24]:
train_sample.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,label,label_multiclass
1250941,5643809,0.000000,1980s USA routs USSR. Winner - Iran. No more e...,0.00,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-23 15:01:56.128046+00,54,NaN,358404,rejected,0,0,0,0,0,0.00,0,4,0,not_toxic
279216,584484,0.000000,I write during election day as ballots are sti...,0.00,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-08 17:30:19.184759+00,53,NaN,150883,approved,0,0,0,1,0,0.00,0,4,0,not_toxic
880431,5197854,0.000000,My 911 will never hit the resale market . Non-...,0.00,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-30 16:06:09.307109+00,54,5189868.0,330092,approved,0,0,0,0,0,0.00,0,4,0,not_toxic
146876,421727,0.000000,"""You can`t fool all the people, all the time.....",0.00,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-08 17:21:33.854424+00,21,420995.0,143205,approved,0,0,0,1,0,0.00,0,4,0,not_toxic
1205377,5588980,0.626667,This man here is a big fan of sexual assault!,0.04,0.013333,0.026667,0.4,0.053333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-07-13 22:36:36.407757+00,102,5588786.0,355090,approved,0,0,0,1,0,0.44,6,75,1,very toxic


In [332]:
# train_drop = train_sample[train_sample.comment_text.apply(exl.tokenizer).apply(len) <= 0].index
# train_drop

Int64Index([  59512, 1793038, 1184410, 1580427,  724850,  734725, 1557245,
            1342916, 1382702, 1211128, 1229590, 1678594,  575017,  262428,
             889620,  912764,  247641, 1121494,  697462, 1397745, 1211128,
             856821, 1741764, 1516224, 1239748, 1089317,  823409,   35976,
             633848, 1522947, 1312149,  575017,  579673, 1211512,  751400,
             298847, 1325795,  955811, 1726165,  833213, 1522947,  422750,
            1520125, 1434204, 1387919, 1073797, 1520125, 1239748,  134359,
             301923,  777401,   91317,  512414,  603073,  304749, 1068988,
             973659,  813976,  925937,  503808, 1483813, 1245547],
           dtype='int64')

In [0]:
# train_sample = train_sample.drop(train_drop, axis = 0)

In [0]:
val_samples = val_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [28]:
val_samples.shape

(57751, 47)

In [333]:
# val_drop = val_samples[val_samples.comment_text.apply(exl.tokenizer).apply(len) <= 0].index
# val_drop

Int64Index([1142416,  465369, 1108078,  239717,  545435, 1785002, 1439732,
            1431356, 1670979,  674520,  134403, 1033370, 1049276,  296895,
             672091, 1158704,     316,  609516],
           dtype='int64')

In [0]:
# val_samples = val_samples.drop(val_drop, axis = 0)

In [0]:
test_samples = test_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [334]:
# test_drop = test_samples[test_samples.comment_text.apply(exl.tokenizer).apply(len) <= 0].index
# test_drop

Int64Index([1622847, 1394777,  791534,  461805, 1075192, 1289137, 1651804,
            1178279, 1569896,   46991,  969825],
           dtype='int64')

In [0]:
# test_samples = test_samples.drop(test_drop, axis = 0)

In [0]:
# H/T for the torchtext dataframe-based-Dataset hack to: 
# https://stackoverflow.com/questions/52602071/dataframe-as-datasource-in-torchtext
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
        examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
            string is a field name, and the Field is the associated field.
        filter_pred (callable or None): use only exanples for which
            filter_pred(example) is true, or use all examples if None.
            Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

                
class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""

    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
#             print(key)
#             print(field)
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [0]:
def tokenizer(text, stop_ws=exl.stops, stemmer=None, str_output=False):

    t = text.replace("-", " ").split(" ")
    t = [w.strip(string.punctuation) for w in t]

    if stop_ws:
        t = [w.lower() for w in t if w not in stop_ws]

    if stemmer:
        t = [stemmer.stem(w) for w in t]

    if t == []:
      return ['<blank>']
    if str_output:
        return ' '.join(t)
    else:
        return t

In [0]:
from torchtext.data import Field, BucketIterator

TEXT = Field(tokenize=tokenizer, lower=True, include_lengths = True)
LABEL = Field(sequential=False, unk_token=None, 
              dtype=torch.float)

In [0]:
train_ds = DataFrameDataset(train_sample, fields = {'label' : LABEL, 
                                                    'comment_text': TEXT})
valid_ds = DataFrameDataset(val_samples, fields = {'label' : LABEL, 
                                                   'comment_text': TEXT})
test_ds = DataFrameDataset(test_samples, fields = { 'label' : LABEL, 
                                                   'comment_text' : TEXT})

In [487]:
train_ds.fields

{'comment_text': <torchtext.data.field.Field at 0x7f241e3de0b8>,
 'label': <torchtext.data.field.Field at 0x7f241e3de0f0>}

In [488]:
valid_ds.fields

{'comment_text': <torchtext.data.field.Field at 0x7f241e3de0b8>,
 'label': <torchtext.data.field.Field at 0x7f241e3de0f0>}

In [0]:
TEXT.build_vocab(train_ds, max_size=25000, 
#                  vectors = "glove.840B.300d",
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_ds)

In [490]:
print('len(TEXT.vocab)', len(TEXT.vocab))
# print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

len(TEXT.vocab) 25002


In [491]:
print(TEXT.vocab.stoi['hope'])

200


In [492]:
print(TEXT.vocab.itos[198])

true


In [0]:
# Check Available RAM

# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [0]:
# Clear RAM

# !kill -9 -1

In [0]:
# # Attribution: Adapted from https://towardsdatascience.com/how-to-use-torchtext-for-neural-machine-translation-plus-hack-to-make-it-5x-faster-77f3884d95
# import torch
# from torchtext import data
# import numpy as np

# global max_text_in_batch

# def batch_size_fn(new, count, sofar):
#     "Keep augmenting batch and calculate total number of tokens + padding."
#     global max_text_in_batch
#     if count == 1:
#         max_text_in_batch = 0
#     max_text_in_batch = max(max_text_in_batch, len(new.comment_text) + 2)
#     text_elements = count * max_text_in_batch
#     return text_elements


# class EfficientIterator(data.Iterator):
#     def create_batches(self):
#         if self.train:
#             def pool(d, random_shuffler):
#                 for p in data.batch(d, self.batch_size * 100):
#                     p_batch = data.batch(
#                         sorted(p, key=self.sort_key),
#                         self.batch_size, self.batch_size_fn)
#                     for b in random_shuffler(list(p_batch)):
#                         yield b
#             self.batches = pool(self.data(), self.random_shuffler)
            
#         else:
#             self.batches = []
#             for b in data.batch(self.data(), self.batch_size,
#                                           self.batch_size_fn):
#                 self.batches.append(sorted(b, key=self.sort_key))

In [493]:
VOCAB_SIZE = len(TEXT.vocab)
BATCH_SIZE = 1300
VOCAB_SIZE

25002

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

train_iter, val_iter, test_iter = BucketIterator.splits(
    (train_ds, valid_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)

In [495]:
device

device(type='cuda', index=0)

In [0]:
# train_iter = EfficientIterator(
#     train_ds, batch_size=1300, 
#     device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
#     repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
#     train=True, shuffle=True)


In [496]:
batch = next(iter(train_iter))
print(batch.label)
print(batch.label.size())
print(batch.comment_text)
print(batch.comment_text[0].size())

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
torch.Size([1300])
(tensor([[    0,     0,   133,  ...,   952,   143,  4746],
        [    8,    61, 16130,  ...,   555,    64,     3],
        [22173,  1299,  3039,  ...,   230,  3614,   691],
        ...,
        [    0,    77,   110,  ...,     7,    80, 17413],
        [10131,  1000,    62,  ...,  5331,  3881,  3139],
        [ 2852,  1319,  3039,  ...,  3984,  8353,  6362]], device='cuda:0'), tensor([9, 9, 9,  ..., 9, 9, 9], device='cuda:0'))
torch.Size([9, 1300])


In [497]:
val_batch = next(iter(val_iter))
print(val_batch.label)
print(val_batch.label.size())
print(val_batch.comment_text)
print(val_batch.comment_text[0].size())

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
torch.Size([1300])
(tensor([[  70,   31,    0,  ..., 2697, 2192, 1433],
        [  57, 5855,    0,  ...,    1,    1,    1]], device='cuda:0'), tensor([2, 2, 2,  ..., 1, 1, 1], device='cuda:0'))
torch.Size([2, 1300])


In [498]:
test_batch = next(iter(test_iter))
print(test_batch.label)
print(test_batch.label.size())
print(test_batch.comment_text)
print(test_batch.comment_text[0].size())

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
torch.Size([1300])
(tensor([[ 1197, 16468,    28,  ...,  2371,     0,   890],
        [ 4804,    28,  1190,  ...,     1,     1,     1]], device='cuda:0'), tensor([2, 2, 2,  ..., 1, 1, 1], device='cuda:0'))
torch.Size([2, 1300])


In [0]:
# val_iter = EfficientIterator(
#     valid_ds, batch_size=1300, 
#     device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
#     repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
#     train=True, shuffle=True)


In [0]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, pad_idx, batch_size=1,
                 embed_dim=6, weight_decay=0, optimizer_fcn='Adam',
                 learning_rate=1e-3, num_layers=2, dropout=0.05, 
                 num_classes=1, bidirectional=True):
        super(LSTMModel, self).__init__()
        nn.Module.__init__(self)
#         TextData.__init__(self, X_data)
        self.vocab_size = VOCAB_SIZE
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.output_dim = num_classes
        self.loss_fcn = nn.NLLLoss()
        self.weight_decay = weight_decay
        self.learning_rate = learning_rate
        self.highest_f1 = -math.inf
        self.pad_idx = pad_idx

        # Layer 1: Embedding Layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_dim, padding_idx=pad_idx)

        # Layer 2: LSTM Layer
        self.lstm = nn.LSTM(input_size = self.embed_dim, 
                            hidden_size = self.hidden_dim,
                            num_layers = self.num_layers, 
                            dropout = self.dropout,
                            bidirectional=bidirectional) 
#                             batch_first=True)

        # Layer 3 (Output Layer): Linear
        self.linear = nn.Linear(self.hidden_dim * 2, self.output_dim)
    
        # Dropout Layer  
        self.dropout_layer = nn.Dropout(self.dropout)

        # define optimizer
        if optimizer_fcn == 'Adam':
            self.optimizer = optim.Adam(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'RMSprop':
            self.optimizer = optim.RMSprop(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'SDG':
            self.optimizer = optim.SGD(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
            

    def forward(self, input_seq, input_len):
        # input_seq shape is [batch_max_input_len, batch_size]
        embed_out = self.embedding(input_seq)
        embed_out = self.dropout_layer(embed_out)

        # embed_out shape is [batch_max_input_len, batch_size, embed_dim]

        
        # tell LSTM layer to focus on non-padded elements by packing embeddings
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embed_out, input_len)
        
        packed_lstm_out, (hn, cn) = self.lstm(packed_embedded)
        # hn shape is [num_layers * num_directions, batch_size, hidden_dim]
        
        # unpack sequences into tensors if need them
        # out, out_len = nn.utils.rnn.pad_packed_sequence(packed_lstm_out)
        # out shape is [batch_max_input_len, batch_size, hidden_dim * num directions]
        
        #concat final forward and backward hidden layers to apply any dropout
        hn = self.dropout_layer(torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1))
        # hn shape is [batch size, hid dim * num directions]
        
#         out = F.log_softmax(self.linear(hn.squeeze(0)), dim=self.output_dim) 
        out = F.log_softmax(self.linear(hn), dim=1)
        # out shape is [batch_size, outpu_dim]
        

        return out


In [0]:
NUM_EPOCHS = 6
hist_lstm = np.zeros(NUM_EPOCHS)
model_name="20_pct_dataset"

In [0]:
lstm_model = LSTMModel(vocab_size = VOCAB_SIZE, hidden_dim=100, num_layers=1, 
#                        embed_dim=300, 
                       embed_dim=200, 
                       batch_size=train_iter.batch_size,
                       dropout=0, num_classes=2, 
                       pad_idx=TEXT.vocab.stoi[TEXT.pad_token])

In [502]:
torch.cuda.is_available()

True

In [0]:
if torch.cuda.is_available():
    lstm_model = lstm_model.cuda()
    lstm_model.loss_fcn = lstm_model.loss_fcn.cuda()

In [504]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 200])


In [505]:
lstm_model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.9358,  0.2389, -0.9769,  ..., -0.2504,  0.6943,  0.7544],
        [-1.0624,  1.3300, -0.2345,  ..., -0.3028,  0.8864, -0.8677],
        [ 0.2680,  0.3603, -0.3320,  ...,  0.1777,  0.2236,  0.0142],
        ...,
        [-0.7283,  0.4809, -0.4850,  ...,  0.0470, -0.0370, -0.3155],
        [-0.0343,  0.4507,  0.7884,  ..., -0.1092,  0.3405, -0.0644],
        [-0.0034,  0.2970, -0.2874,  ...,  0.5814,  0.1632,  0.2532]],
       device='cuda:0')

In [506]:
# let model learn from unknown token, while leaving padding as zeros
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

lstm_model.embedding.weight.data[UNK_IDX] = torch.zeros(lstm_model.embed_dim)
lstm_model.embedding.weight.data[lstm_model.pad_idx] = torch.zeros(lstm_model.embed_dim)

print(lstm_model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2680,  0.3603, -0.3320,  ...,  0.1777,  0.2236,  0.0142],
        ...,
        [-0.7283,  0.4809, -0.4850,  ...,  0.0470, -0.0370, -0.3155],
        [-0.0343,  0.4507,  0.7884,  ..., -0.1092,  0.3405, -0.0644],
        [-0.0034,  0.2970, -0.2874,  ...,  0.5814,  0.1632,  0.2532]],
       device='cuda:0')


In [0]:
def train_clf(model, iterator):
    running_loss = 0
    epoch_f1 = 0
    epoch_nontarget_f1 = 0
    
    model.train()

    for batch in iterator:
        if len(batch) != BATCH_SIZE:
            continue
          
        model.optimizer.zero_grad()

        seq, seq_lengths = batch.comment_text
        y = batch.label.long()

        if torch.cuda.is_available():
            seq = seq.cuda() 
            y = y.cuda()
            seq_lengths = seq_lengths.cuda()

        preds = model.forward(seq, seq_lengths)

        loss = model.loss_fcn(input=preds.squeeze(1), 
                                     target=y)

        y_true = [single_tensor.item() for single_tensor in y]

        indices = torch.argmax(preds, 1)
        y_pred = [single_tensor.item() for single_tensor in indices.long()]

        f1_toxic = f1_score(y_true, y_pred, pos_label=1)
        f1_nontoxic = f1_score(y_true, y_pred, pos_label=0)

        loss.backward()

        running_loss += loss.item()
        epoch_f1 += f1_toxic.item()
        epoch_nontarget_f1 += f1_nontoxic.item()

        model.optimizer.step()
        
    last_mean_error = np.mean(np.abs(loss.item()))
    return running_loss / len(iterator), epoch_f1 / len(iterator),  epoch_nontarget_f1 / len(iterator), last_mean_error


In [0]:
def evaluate_clf(model, iterator):
    
    running_loss = 0
    epoch_f1 = 0
    epoch_nontarget_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            seq, seq_lengths = batch.comment_text
            
            y = batch.label.long()
            
            predictions = model.forward(seq, seq_lengths)
            
            loss = model.loss_fcn(predictions.squeeze(1), y)
            
            val_true = [single_tensor.item() for single_tensor in y]

            indices = torch.argmax(predictions, 1)
            val_pred = [single_tensor.item() for single_tensor in indices]

            f1_toxic = f1_score(val_true, val_pred, pos_label=1)
            f1_nontoxic = f1_score(val_true, val_pred, pos_label=0)
        
            recall_toxic = recall_score(val_true, val_pred, pos_label=1)
            recall_nontoxic = recall_score(val_true, val_pred, pos_label=0)
            
            precision_toxic = precision_score(val_true, val_pred, pos_label=1)
            precision_nontoxic = precision_score(val_true, val_pred, pos_label=0)
            
            accuracy = accuracy_score(val_true, val_pred)
            
            try:
                auc_roc = roc_auc_score(val_true, val_pred)
            except:
                auc_roc = -1

            running_loss += loss.item()
            epoch_f1 += f1_toxic.item()
            epoch_nontarget_f1 += f1_nontoxic.item()
  
            metrics = {
                "target": {"f1": round(f1_toxic * 100, 5),
                           "precision": round(precision_toxic * 100, 5),
                           "recall": round(recall_toxic * 100, 5)
                          },
                "nontarget": {"f1": round(f1_nontoxic * 100, 5),
                           "precision": round(precision_nontoxic * 100, 5),
                           "recall": round(recall_nontoxic * 100)
                          },
                "overall": {"accuracy": round(accuracy * 100, 5),
                           "auc_roc": round(auc_roc * 100, 5)}
            }
   
        
    return running_loss / len(iterator), epoch_f1 / len(iterator),  epoch_nontarget_f1 / len(iterator), metrics

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def loop_epochs(model, train_iter, val_iter, num_epochs, outfile=None, model_prefix=None):  
    loop_start = time.time()
    if outfile:
        f = open(outfile, "a")
    else:
        f = None
      
    best_valid_loss = float('inf')
    last_mean_error = -math.inf

    thresholds= {
            5000: 1000,
            1000: 100,
            100: 10,
            20: 5,
            10: 3
        }
    metric_results = dict()

    for epoch in range(num_epochs):

        start_time = time.time()

        print(f"Epoch {epoch}:\nStarting training...", file=f)
        train_loss, target_f1, nontarget_f1, epoch_mean_error = train_clf(model, train_iter)

        end_train = time.time()
        train_mins, train_secs = epoch_time(start_time, end_train)

        print(f"Starting evaluation...({train_mins:02}:{train_secs:02} elapsed)", file=f)
        valid_loss, val_target_f1, val_nontarget_f1, metrics = evaluate_clf(model, val_iter)

        end_eval = time.time()
        eval_mins, eval_secs = epoch_time(end_train, end_eval)
        print(f"Finished evaluation...({eval_mins:02}:{eval_secs:02} elapsed)", file=f)

        epoch_mins, epoch_secs = epoch_time(start_time, end_eval)

        metric_results[epoch] = metrics

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            savestate_file = f'{model_prefix}_model_state.pt'
            torch.save(model.state_dict(), savestate_file)

        print(f'Epoch: {epoch+1:02} | Total Epoch Time: {epoch_mins}m {epoch_secs}s', file=f)
        print(f'\tNegative Log Linear Loss:  Loss: {train_loss:.6f} | Target F1: {target_f1*100:.2f}% | Nontarget F1: {nontarget_f1*100:.2f}%', file=f)
        print(f'\tVal. Loss: {valid_loss:.6f} | Target F1: {val_target_f1*100:.2f}% | Nontarget F1: {val_nontarget_f1*100:.2f}%', file=f)
        
        
    loop_mins, loop_secs = epoch_time(loop_start, end_eval)
    print(f"Total Time: {loop_mins:02}:{loop_secs:02} elapsed", file=f)
    print('', file=f)
    print(metric_results, file=f)
    print('', file=f)
    print('', file=f)
    
    if outfile:
        send_to_bucket(filename=outfile)
    send_to_bucket(filename=savestate_file, directory='model_states')

   

In [0]:
loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile='20pct_imbalanced.txt', model_prefix='20pct_imbalanced')

In [553]:
project_id = 'tribal-monolith-242216'
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey



In [0]:
def send_to_bucket(filename, bucket_name="no-hate", directory=None):
    if directory:
        !gsutil cp /content/{filename} gs://{bucket_name}/{directory}/
    else:
        !gsutil cp /content/{filename} gs://{bucket_name}/
        
    

In [557]:
send_to_bucket(filename='20pct_imbalanced_model_state.pt')

Copying file:///content/20pct_imbalanced_model_state.pt [Content-Type=application/octet-stream]...
\
Operation completed over 1 objects/20.0 MiB.                                     


In [0]:
torch.cuda.empty_cache()

In [0]:
train_set, val_set, test_set = exl.get_samples(train_full, proportion=0.2, 
                                                  train_test_ratio=(1-test_ratio))

In [0]:
import cloudstorage

In [0]:
cloudstorage.

In [0]:
import io
import gcsfs
from io import BytesIO
import pickle

def pickle_to_gcs(df, filename, bucket_name="no-hate"):
    pickle_buffer = io.BytesIO()
    fs = gcsfs.GCSFileSystem(project=project_id)
    df.to_pickle(filename)
    
    with fs.open(f"{bucket_name}/{filename}", "wb") as handle:    
        pickle.dump(df, handle)
        
    
def load_pickle_from_gcs(filename, bucket_name="no-hate"):
    fs = gcsfs.GCSFileSystem(project=project_id)    
    with fs.open(f"{bucket_name}/{filename}", "rb") as handle:
        df = pickle.load(handle)
    return df

In [0]:
pickle_to_gcs(test_set, '20pct_test_set.pkl')

In [599]:
print(test_set.shape, test_set.comment_text.isna().sum())

(180958, 47) 0


In [0]:
check = load_pickle_from_gcs('20pct_test_set.pkl')

In [598]:
print(check.shape, check.comment_text.isna().sum())

(180958, 47) 0


In [0]:
def initiate_model(train_df, val_df, batch_size=BATCH_SIZE):
    train_ds = DataFrameDataset(train_df, fields = {'label' : LABEL, 
                                                        'comment_text': TEXT})
    valid_ds = DataFrameDataset(val_df, fields = {'label' : LABEL, 
                                                       'comment_text': TEXT})
#     test_ds = DataFrameDataset(test_df, fields = { 'label' : LABEL, 
#                                                        'comment_text' : TEXT})
    print("Torchtext Datasets created...")
    TEXT.build_vocab(train_ds, max_size=25000, 
#                  vectors = "glove.840B.300d",
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)
    LABEL.build_vocab(train_ds)
    
    print("Torchtext vocabulary cerated...")
    
    VOCAB_SIZE = len(TEXT.vocab)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

    train_iter, val_iter = BucketIterator.splits(
    (train_ds, valid_ds), 
    batch_size = batch_size,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)
    
    print("Torchtext iterators cerated...")
    
    lstm_model = LSTMModel(vocab_size = VOCAB_SIZE, hidden_dim=100, num_layers=1, 
                           embed_dim=200, 
                           batch_size=train_iter.batch_size,
                           dropout=0, num_classes=2, 
                           pad_idx=TEXT.vocab.stoi[TEXT.pad_token])
    
    print("Model created...")
    
    if torch.cuda.is_available():
        lstm_model = lstm_model.cuda()
        lstm_model.loss_fcn = lstm_model.loss_fcn.cuda()
        
    pretrained_embeddings = TEXT.vocab.vectors
    lstm_model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    lstm_model.embedding.weight.data[UNK_IDX] = torch.zeros(lstm_model.embed_dim)
    lstm_model.embedding.weight.data[lstm_model.pad_idx] = torch.zeros(lstm_model.embed_dim)
    print("Pre-trained embeddings applied...")
    
    return lstm_model, train_iter, val_iter


In [0]:
rebalance_dict = {0: 35, 1: 50, 2: 60, 3: 65, 4:75, 5: 'random'}
data_proportions = [0.2, 0.4, 0.6, 0.8]

In [624]:


prepared_35, prepared_50, prepared_60, prepared_65, prepared_75, random_df = exl.rebalance_data(train_set)

Rebalance Ratio: 0.35, 101006 toxic samples out of 288591
Rebalance Ratio: 0.5, 144295 toxic samples out of 288591
Rebalance Ratio: 0.6, 173154 toxic samples out of 288591
Rebalance Ratio: 0.65, 187584 toxic samples out of 288591
Rebalance Ratio: 0.75, 216443 toxic samples out of 288591
Rebalanced dfs created...


In [621]:
test_ratio=0.2

for i, df in enumerate([prepared_35, prepared_50, prepared_60, prepared_65, prepared_75, random_df]):
    model_name=f'20pct_model_{rebalance_dict[i]}toxic'
    print(f'Model: {model_name}')
    val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), random_state=1008 )
    
    print(f"Validation sample_created...")
    
    lstm_model, train_iter, val_iter = initiate_model(df, val_sample_df, batch_size=BATCH_SIZE)
    
    print(f"Beginning training and evaluation loops...")

    loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile=f'{model_name}.txt', model_prefix=model_name)


Model: 20pct_model_35toxic
Validation sample_created...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Copying file:///content/20pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Copying file:///content/20pct_model_35toxic_model_state.pt [Content-Type=application/octet-stream]...
\
Operation completed over 1 objects/20.0 MiB.                                     
Model: 20pct_model_50toxic
Validation sample_created...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/20pct_model_50toxic.txt [Content-Type=text/plain]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Copying file:///content/20pct_model_50toxic_model_state.pt [Content-Type=application/octet-

In [0]:
for proportion in data_proportions[1:]:
    train_set, val_set, test_set = exl.get_samples(train_full, proportion=proportion, 
                                                  train_test_ratio=(1-test_ratio))
    pickle_to_gcs(test_set, f'{proportion*100:.0f}pct_test_set.pkl')
        
    rebalanced_dfs = exl.rebalance_data(train_set)
    for i, df in enumerate(rebalanced_dfs):
        
        model_name=f'{proportion*100:.0f}pct_model_{rebalance_dict[i]}toxic'
        
        print(f'Model: {model_name}')
        
        pickle_to_gcs(df, f'{model_name}_train.pkl')
        
        val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), random_state=1008 )
        
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl',)

        print(f"Validation sample_created...")

        lstm_model, train_iter, val_iter = initiate_model(df, val_sample_df, batch_size=BATCH_SIZE)

        print(f"Beginning training and evaluation loops...")

        loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
                outfile=f'{model_name}.txt', model_prefix=model_name)


Retrieving training samples at proportion 0.4
Rebalance Ratio: 0.35, 202297 toxic samples out of 577993
Rebalance Ratio: 0.5, 288996 toxic samples out of 577993
Rebalance Ratio: 0.6, 346795 toxic samples out of 577993
Rebalance Ratio: 0.65, 375695 toxic samples out of 577993
Rebalance Ratio: 0.75, 433494 toxic samples out of 577993
Rebalanced dfs created...
Model: 40.0pct_model_35toxic
Validation sample_created...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
